In [9]:
# port AML_CW2 to notebook file format for web browsing and pdf conversion

In [14]:
# machine learning libraries
import pandas as pd
import scipy.io
import classifiers as clf
from sklearn.feature_extraction.text import TfidfVectorizer

# file handling libraries
import glob
import os
from os.path import *

In [17]:
current_dir = os.getcwd()

data_file_name = 'spham_data.mat'

if not exists(join(current_dir + data_file_name)):

    print("Creating data.mat file...")
              
    file_paths = [current_dir + '/data/' + folder + '*.txt' for folder in ["spam/", "ham/", "test/"]]

    spam = glob.glob(file_paths[0])
    ham  = glob.glob(file_paths[1])
    test = glob.glob(file_paths[2])
              
    # Construct vectorizer for TF-IDF feature extraction 
    vectorizer = TfidfVectorizer(input='filename',
                                 lowercase = True, 
                                 stop_words="english",
                                 encoding='latin-1', 
                                 min_df = 8)
    
    spham = spam + ham
    tr = vectorizer.fit_transform(spham) # fit_transform returns a term-document matrix object
    ts = vectorizer.transform(test) 
              
    dicy = {
        'training_labels': [1] * len(spam) + [0] * len(ham),
        'training_data': tr, 
        'test_data': ts
    }

    # Saving dictionary as variable to reuse if necessary
    scipy.io.savemat('data.mat', dicy)

# Loading data
data = scipy.io.loadmat(data_file_name)   

Creating data.mat file...


In [30]:
svm_prediction = clf.svm(data, 0.1)
print(svm_prediction.shape)
svm_prediction

(5857,)


array([0, 1, 0, ..., 1, 0, 0])

In [ ]:
clf.setX(data['training_data'])



# Random Forest
# ntrees = The number of trees in the forest
ntrees = 100
res_rforest = clf.rndForest(data, ntrees)

# Logistic Regression
# C = Inverse of regularization strength
C = 1.0
res_lg = clf.logReg(data, C)

# Extreme Gradient Boost
# md = Max depth
# ss = Subsample ratio of training instance
# cs = Subsample ratio of columns when constructing each tree
md = 1
ss = 0.8
cs = 0.8
res_xgb = clf.xgboost(data, md, ss, cs)

# Linear Discriminant Analysis
res_lda = clf.lda(data)

# Quadratic Discriminant Analysis
res_qda = clf.qda(data)

# Ada Boost
# ne = Maximum number of estimators at which boosting is terminated
ne = 50
res_ada = clf.adaboost(data, ne)

# Building array of results
results = [res_svm, res_rforest, res_lg, res_xgb, res_lda, res_qda, res_ada]

print("Writing Results...")
# Saving results
createCSV(res_svm, 'resSVM')
createCSV(res_rforest, 'resRndForest')
createCSV(res_lg, 'resLogReg')
createCSV(res_xgb, 'resXGBoost')
createCSV(res_lda, 'resLDA')
createCSV(res_qda, 'resQDA')
createCSV(res_ada, 'resADA')

# 0 as second parameter to evaluat 'Ham' accuracy
printRes(results, 0) 
print(len(res_svm))
print("\n ****** Finished ******")